In [1]:
import kabuki
import sys
import os
import hddm
import matplotlib.pyplot as plt
import pickle
from joblib import Parallel,delayed
from glob import glob

from kabuki.analyze import gelman_rubin
import arviz as az
import numpy as np
import pandas as pd
from patsy import dmatrix 


from sklearn.preprocessing import StandardScaler

/opt/conda/lib/python3.8/site-packages/IPython/parallel.py:12: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  warn("The `IPython.parallel` package has been deprecated since IPython 4.0. "


In [2]:
scripts_dir = '/home/jovyan/scripts'
sys.path.append(scripts_dir)


from HDDMarviz import HDDMarviz
from InferenceDataFromHDDM import InferenceDataFromHDDM
from plot_ppc_by_cond import plot_ppc_by_cond

In [3]:
# data preprocess
data = hddm.load_csv('./data_beh_eeg.csv',sep='\t')

df = data
df['rt']=df['rt']+0.2
df['response']=df['correct']

df = hddm.utils.flip_errors(df)

In [4]:
df['cpp_slopes']=StandardScaler().fit_transform(df['cpp_slopes'].to_numpy().reshape(-1,1))

df['cpp_amplitude']=StandardScaler().fit_transform(df['cpp_amplitude'].to_numpy().reshape(-1,1))

df['cpp_peak']=StandardScaler().fit_transform(df['cpp_peak'].to_numpy().reshape(-1,1))

In [5]:
df.dropna(subset=['rt','cpp_amplitude','cpp_peak','cpp_slopes'],inplace=True)
df = df.reset_index(drop=True)

In [6]:
df

,condition,coherence,prioritized,subj_idx,rt,key_press,correct,run,stim,n170_amplitude,n170_slopes,baseline,cpp_amplitude,cpp_slopes,cpp_var,cpp_peak,cpp_peak_latency,response
0,1.0,high,yes,sub-001,0.729206,face,1.0,1,face,-0.000006,2.340534e-07,-7.629888e-22,0.648190,2.001235,1.560474e-11,0.871625,0.417096,1.0
1,2.0,high,no,sub-001,0.719501,car,1.0,1,car,NaN,NaN,9.063582e-24,1.225435,-0.174603,7.937925e-12,1.264771,0.346845,1.0
2,2.0,high,no,sub-001,0.811204,face,1.0,1,face,0.000002,-9.800590e-07,-5.932526e-23,0.160142,0.312037,8.048211e-12,0.486489,0.489329,1.0
3,1.0,high,yes,sub-001,0.665089,car,1.0,1,car,NaN,NaN,2.175260e-22,-1.837984,0.491876,2.988462e-12,-1.845775,0.360793,1.0
4,1.0,high,yes,sub-001,0.567166,car,1.0,1,car,NaN,NaN,-6.591696e-23,-1.333245,1.411188,1.284015e-11,-1.297068,0.262869,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4554,3.0,low,yes,sub-017,0.509091,car,1.0,2,car,NaN,NaN,4.268123e-22,0.095669,0.163020,7.588991e-12,0.036959,0.161825,1.0
4555,3.0,low,yes,sub-017,0.461170,car,1.0,2,car,NaN,NaN,-2.389490e-23,0.654608,-0.030734,9.850273e-13,-0.179208,0.088514,1.0
4556,3.0,low,yes,sub-017,-0.471483,car,0.0,2,face,-0.000009,1.896555e-07,5.586462e-22,-0.146989,-0.062717,7.014867e-12,-0.269449,0.092968,0.0
4557,4.0,low,no,sub-017,0.483706,car,1.0,2,car,NaN,NaN,-1.977509e-22,-0.003302,-0.105106,1.355688e-12,-0.487029,0.154019,1.0


In [9]:
def run_m15(id, df=None, samples=None, burn=None,thin=1, save_name=None): 
    
    dbname = save_name + '_chain_%i.db'%id 

    mname  = save_name + '_chain_%i'%id    
   
    v_reg={'model': "v~1+C(coherence, Treatment('low'))+cpp_slopes+cpp_slopes:C(coherence, Treatment('low'))", 'link_func': lambda x: x}
    
    t_reg={'model': "t~1+C(prioritized, Treatment('no'))", 'link_func':   lambda x: x}
    
    m=hddm.HDDMRegressor(df ,[v_reg, t_reg] ,include=['z'],p_outlier=0.05,keep_regressor_trace=True, group_only_regressors = False)  
    
    m.find_starting_values()
    
    m.sample(samples, burn=burn, dbname=dbname, thin=thin,db='pickle') 
    
    m.save(mname)
    
    return m

In [11]:
m15 = Parallel(n_jobs = 4)(delayed(run_m15)(id = i, df = df, samples=4000,burn = 2000,thin=1, save_name = 'temp/m15' ) for i in range(4))

In [12]:
def run_m16(id, df=None, samples=None, burn=None,thin=1, save_name=None): 
    
    dbname = save_name + '_chain_%i.db'%id 

    mname  = save_name + '_chain_%i'%id    
    
    v_reg={'model': "v~1+C(coherence, Treatment('low'))+cpp_amplitude+cpp_amplitude:C(coherence, Treatment('low'))", 'link_func': lambda x: x}
    
    t_reg={'model': "t~1+C(prioritized, Treatment('no'))", 'link_func':   lambda x: x}

    m = hddm.HDDMRegressor(df, [v_reg, t_reg] ,include=['z'],keep_regressor_trace=True, group_only_regressors = False)
 
    
    m.find_starting_values()
    
    m.sample(samples, burn=burn, dbname=dbname, thin=thin,db='pickle') 
    
    m.save(mname)
    
    return m

In [13]:
m16 = Parallel(n_jobs = 4)(delayed(run_m16)(id = i, df = df, samples=4000,burn = 2000,thin=1, save_name = 'temp/m16' ) for i in range(4))

In [14]:
def run_m17(id, df=None, samples=None, burn=None,thin=1, save_name=None): 
    
    dbname = save_name + '_chain_%i.db'%id 

    mname  = save_name + '_chain_%i'%id    
      
    v_reg={'model': "v~1+C(coherence, Treatment('low'))+cpp_peak+cpp_peak:C(coherence, Treatment('low'))", 'link_func': lambda x: x}
    
    t_reg={'model': "t~1+C(prioritized, Treatment('no'))", 'link_func':   lambda x: x}

    m = hddm.HDDMRegressor(df, [v_reg, t_reg] ,include=['z'],keep_regressor_trace=True, group_only_regressors = False)
 
    
    m.find_starting_values()
    
    m.sample(samples, burn=burn, dbname=dbname, thin=thin,db='pickle') 
    
    m.save(mname)
    
    return m

In [15]:
m17 = Parallel(n_jobs = 4)(delayed(run_m17)(id = i, df = df, samples=4000,burn = 2000,thin=1, save_name = 'temp/m17' ) for i in range(4))

In [16]:
# data preprocess
data = hddm.load_csv('./data_beh_eeg.csv',sep='\t')

df = data
df['rt']=df['rt']+0.2
df['response']=df['correct']

df = hddm.utils.flip_errors(df)


for subj in df.subj_idx.unique():
    for cond in df.coherence.unique():
        df.loc[(df['subj_idx']==subj)&(df['coherence']==cond),'cpp_slopes']=df.loc[df['subj_idx']==subj,'cpp_slopes'].mean()

        df.loc[(df['subj_idx']==subj)&(df['coherence']==cond),'cpp_amplitude']=df.loc[df['subj_idx']==subj,'cpp_amplitude'].mean()

        df.loc[(df['subj_idx']==subj)&(df['coherence']==cond),'cpp_peak']=df.loc[df['subj_idx']==subj,'cpp_peak'].mean()


df['cpp_slopes']=StandardScaler().fit_transform(df['cpp_slopes'].to_numpy().reshape(-1,1))

df['cpp_amplitude']=StandardScaler().fit_transform(df['cpp_amplitude'].to_numpy().reshape(-1,1))

df['cpp_peak']=StandardScaler().fit_transform(df['cpp_peak'].to_numpy().reshape(-1,1))

df.dropna(subset=['rt','cpp_amplitude','cpp_peak','cpp_slopes'],inplace=True)
df = df.reset_index(drop=True)

In [18]:
def run_m18(id, df=None, samples=None, burn=None,thin=1, save_name=None): 
    
    dbname = save_name + '_chain_%i.db'%id 

    mname  = save_name + '_chain_%i'%id    
   
    v_reg={'model': "v~1+C(coherence, Treatment('low'))+cpp_slopes+cpp_slopes:C(coherence, Treatment('low'))", 'link_func': lambda x: x}
    
    t_reg={'model': "t~1+C(prioritized, Treatment('no'))", 'link_func':   lambda x: x}
    
    m=hddm.HDDMRegressor(df ,[v_reg, t_reg] ,include=['z'],keep_regressor_trace=True, group_only_regressors = False)  
    
    m.find_starting_values()
    
    m.sample(samples, burn=burn, dbname=dbname, thin=thin,db='pickle') 
    
    m.save(mname)
    
    return m

In [19]:
m18 = Parallel(n_jobs = 4)(delayed(run_m18)(id = i, df = df, samples=4000,burn = 2000,thin=1, save_name = 'temp/m18' ) for i in range(4))

In [20]:
def run_m19(id, df=None, samples=None, burn=None,thin=1, save_name=None): 
    
    dbname = save_name + '_chain_%i.db'%id 

    mname  = save_name + '_chain_%i'%id    
    
    v_reg={'model': "v~1+C(coherence, Treatment('low'))+cpp_amplitude+cpp_amplitude:C(coherence, Treatment('low'))", 'link_func': lambda x: x}
    
    t_reg={'model': "t~1+C(prioritized, Treatment('no'))", 'link_func':   lambda x: x}

    m = hddm.HDDMRegressor(df, [v_reg, t_reg] ,include=['z'],keep_regressor_trace=True, group_only_regressors = False)
 
    
    m.find_starting_values()
    
    m.sample(samples, burn=burn, dbname=dbname, thin=thin,db='pickle') 
    
    m.save(mname)
    
    return m

In [21]:
m19 = Parallel(n_jobs = 4)(delayed(run_m19)(id = i, df = df, samples=4000,burn = 2000,thin=1, save_name = 'temp/m19' ) for i in range(4))

In [22]:
def run_m20(id, df=None, samples=None, burn=None,thin=1, save_name=None): 
    
    dbname = save_name + '_chain_%i.db'%id 

    mname  = save_name + '_chain_%i'%id    
      
    v_reg={'model': "v~1+C(coherence, Treatment('low'))+cpp_peak+cpp_peak:C(coherence, Treatment('low'))", 'link_func': lambda x: x}
    
    t_reg={'model': "t~1+C(prioritized, Treatment('no'))", 'link_func':   lambda x: x}

    m = hddm.HDDMRegressor(df, [v_reg, t_reg] ,include=['z'],keep_regressor_trace=True, group_only_regressors = False)
 
    
    m.find_starting_values()
    
    m.sample(samples, burn=burn, dbname=dbname, thin=thin,db='pickle') 
    
    m.save(mname)
    
    return m

In [23]:
m20 = Parallel(n_jobs = 4)(delayed(run_m20)(id = i, df = df, samples=4000,burn = 2000,thin=1, save_name = 'temp/m20' ) for i in range(4))

In [24]:
def run_m21(id, df=None, samples=None, burn=None,thin=1, save_name=None): 
    
    dbname = save_name + '_chain_%i.db'%id 

    mname  = save_name + '_chain_%i'%id    
   
    v_reg={'model': "v~1+C(coherence, Treatment('low'))+cpp_slopes+cpp_slopes:C(coherence, Treatment('low'))", 'link_func': lambda x: x}
    
    t_reg={'model': "t~1+C(prioritized, Treatment('no'))", 'link_func':   lambda x: x}
    
    m=hddm.HDDMRegressor(df ,[v_reg, t_reg] ,include=['z','sv','st'],keep_regressor_trace=True, group_only_regressors = False)  
    
    m.find_starting_values()
    
    m.sample(samples, burn=burn, dbname=dbname, thin=thin,db='pickle') 
    
    m.save(mname)
    
    return m

In [25]:
m21 = Parallel(n_jobs = 4)(delayed(run_m21)(id = i, df = df, samples=4000,burn = 2000,thin=1, save_name = 'temp/m21' ) for i in range(4))

In [26]:
def run_m22(id, df=None, samples=None, burn=None,thin=1, save_name=None): 
    
    dbname = save_name + '_chain_%i.db'%id 

    mname  = save_name + '_chain_%i'%id    
    
    v_reg={'model': "v~1+C(coherence, Treatment('low'))+cpp_amplitude+cpp_amplitude:C(coherence, Treatment('low'))", 'link_func': lambda x: x}
    
    t_reg={'model': "t~1+C(prioritized, Treatment('no'))", 'link_func':   lambda x: x}

    m = hddm.HDDMRegressor(df, [v_reg, t_reg] ,include=['z','sv','st'],keep_regressor_trace=True, group_only_regressors = False)
 
    
    m.find_starting_values()
    
    m.sample(samples, burn=burn, dbname=dbname, thin=thin,db='pickle') 
    
    m.save(mname)
    
    return m

In [27]:
m22 = Parallel(n_jobs = 4)(delayed(run_m22)(id = i, df = df, samples=4000,burn = 2000,thin=1, save_name = 'temp/m22' ) for i in range(4))

In [28]:
def run_m23(id, df=None, samples=None, burn=None,thin=1, save_name=None): 
    
    dbname = save_name + '_chain_%i.db'%id 

    mname  = save_name + '_chain_%i'%id    
      
    v_reg={'model': "v~1+C(coherence, Treatment('low'))+cpp_peak+cpp_peak:C(coherence, Treatment('low'))", 'link_func': lambda x: x}
    
    t_reg={'model': "t~1+C(prioritized, Treatment('no'))", 'link_func':   lambda x: x}

    m = hddm.HDDMRegressor(df, [v_reg, t_reg] ,include=['z','sv','st'],keep_regressor_trace=True, group_only_regressors = False)
 
    
    m.find_starting_values()
    
    m.sample(samples, burn=burn, dbname=dbname, thin=thin,db='pickle') 
    
    m.save(mname)
    
    return m

In [29]:
m23 = Parallel(n_jobs = 4)(delayed(run_m23)(id = i, df = df, samples=4000,burn = 2000,thin=1, save_name = 'temp/m23' ) for i in range(4))